In [2]:
import pandas as pd
import re
import numpy as np
import itertools
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import geopandas

## Function Definition

In [8]:
# 取出数据并分层
def extract_and_split(df, district_name):
    if district_name != 'national':
        # 县级PAC
        provs_set = District_dic[district_name]
        provs_id = set(Prov_shp['省代码'][[prov in provs_set for prov in Prov_shp['省']]])
        # print(provs_id)
        county_lst = County_shp['PAC'][[id in provs_id for id in County_shp['省代码']]]
        # print(len(county_lst))
        
        # 提取数据
        district_id = pd.DataFrame({'PAC': county_lst})
        district_df = pd.merge(df, district_id, how = 'right')
    else:
        district_df = df
    
    # 0: high, 1: medium, 2: low
    district_df0 = district_df[district_df['light'] > Edges[1]]
    district_df1 = district_df[[Edges[0] < lit <= Edges[1] for lit in district_df['light']]]
    district_df2 = district_df[district_df['light'] <= Edges[0]]
    
    # 每类图片数量
    print('high: {}, medium: {}, low: {}, total: {}'.format(len(district_df0), len(district_df1), len(district_df2), len(district_df)))
    
    return district_df0, district_df1, district_df2

In [4]:
# 分层聚类函数
def cls_cluster(district_df_lst, cluster_nums):
    cls_df_lst = []
    for i in range(3):
        district_df = district_df_lst[i]
        num = cluster_nums[i]
        
        km = KMeans(n_clusters = num, random_state = 42)
        labels = km.fit_predict(district_df.iloc[:, 0:125])
        if i > 0:
            labels = labels + np.cumsum(cluster_nums)[i-1]
        district_df['cluster_id'] = labels
        
        cls_df_lst.append(district_df)
        
    district_total = pd.concat(cls_df_lst)
    
    cls_count = pd.value_counts(district_total.cluster_id)
    plt.hist(cls_count, bins = 6)  
    plt.show()
    
    return district_total

In [5]:
# generate histogram
def generate_hist(df, cluster_num, PAC = None):
    """
    df.columns: [image_name (image_id), cluster_id, PAC, ...]
    """
    if PAC is None:
        PAC = np.unique(df.PAC)
        
    hist_data = []
    for pac in PAC:
        cur_hist = []
        for i in range(cluster_num):
            cur_hist.append(sum(df.cluster_id[df.PAC == pac] == i))
        
        hist_data.append(cur_hist)
        
    hist_data = pd.DataFrame(hist_data)
    hist_data.columns = [str(i) for i in range(cluster_num)]
    hist_data['PAC'] = PAC
        
    return hist_data

In [6]:
# generate hist
def generate_grid(df):
    """
    df: [image_name (name), cluster_id]
    """
    suffix = r'-2017\.png'
    
    grid_df = df.loc[:, ['name', 'cluster_id']].sort_values(by = 'cluster_id')
    
    # 去掉后缀
    y_x = [re.sub(suffix, '', name) for name in grid_df.name]
    grid_df['name'] = y_x
    
    # 改名
    grid_df.columns = ['y-x', 'cluster_id']
    
    return grid_df

## Readin Data

#### Numeric Features and Image Information

In [ ]:
# 读入原始数据
data = pd.read_csv('Data/2017_features.csv')
data = data.dropna()
#data = data[data['features']!='adsadas']
name_night = pd.read_csv('Data/2017Name_lights.csv')
label = pd.read_csv('Data/PAC_GDP17.csv')
data = pd.merge(data, name_night, how ='left')
data = pd.merge(data, label.loc[:, ['PAC', 'GDP']], how = 'left')

In [ ]:
frontstr = 'Image/2017/'
# 处理原始数据
X = []  # 4096维向量
Name = [] # 图片名
Light = [] # 灯光强度
y = []  # GDP
PAC = []  # 标识
for i in data.index:
    if i % 100 == 0:
        print(i)
        
    # if i >= 100:
    #     break
    
    #try:
    # 4096维向量
    x_i = [float(x) for x in re.split(r', |\[|\]', data['features'].loc[i]) if len(x) > 0]
    x_i = np.array(x_i).reshape(-1, 4096)  # 4096维向量
    X.append(x_i)
    # 灯光强度与图片名称
    str_lst = [s for s in re.split(r', |\[|\]|\(|\)', re.sub("'", '',data['Name'].loc[i])) if len(s) > 0] 
    name_i = [(str_lst[2*i]).replace(frontstr, '') for i in range(len(str_lst)//2)] # 偶数索引
    light_i = [float(str_lst[2*i+1]) for i in range(len(str_lst)//2)]  # 基数索引
    Name.append(name_i)
    Light.append(light_i)
    # GDP
    y.append(data['GDP'].loc[i])
    # PAC
    PAC.append(data['PAC'].loc[i])
    #except:
       # print('error')

# 去空
zeros = [X.index(x) for x in X if x.shape[0] == 0]
X = np.delete(np.array(X), zeros)
Name = np.delete(np.array(Name), zeros)
Light = np.delete(np.array(Light), zeros)
y = np.delete(np.array(y), zeros)
PAC = np.delete(np.array(PAC), zeros)
# 直接排列所有4096维向量，图片名，与灯光强度
X_ = np.concatenate(X)
Name_ = np.concatenate(Name)
Light_ = np.concatenate(Light)

In [ ]:
# 图片编号与PAC编号对应
img_nums = [len(county) for county in X]
img2county = []
for i, num in enumerate(img_nums):
    cur_array = [i]*num
    img2county.append(cur_array)
img2county = np.concatenate(img2county)

#### Geographic Information

In [ ]:
# 地理信息
County_shp = geopandas.GeoDataFrame.from_file('Data/county/县级行政区.shp')
Prov_shp = geopandas.GeoDataFrame.from_file('Data/prov/省级行政区.shp')

In [ ]:
# 地区字典
District_dic = {
    'east': set(['山东省', '江苏省', '浙江省', '安徽省', '江西省', '福建省', '上海市']),
    'south': set(['广东省', '广西壮族自治区', '海南省', '香港特别行政区', '澳门特别行政区']),
    'north': set(['山西省', '河北省', '北京市', '天津市', '内蒙古自治区']),
    'mid': set(['湖北省', '河南省', '湖南省']),
    'northwest': set(['青海省', '宁夏回族自治区', '陕西省', '甘肃省', '新疆维吾尔自治区']),
    'southwest': set(['贵州省', '云南省', '重庆市', '四川省', '西藏自治区']),
    'northeast': set(['吉林省', '黑龙江省', '辽宁省']) 
}
# 分类光强
Edges = [0.02, 3]

## Dimensional Reduction

In [ ]:
# 不做标准化处理
pca = PCA(n_components = 0.80) 
pca.fit(X_)
reduced_X = pca.transform(X_)

In [ ]:
reduced_X.shape  # (254637, 125) in our experiments

In [ ]:
km_df = pd.DataFrame(reduced_X)
km_df['cluster_id'] = np.repeat(0, reduced_X.shape[0])
km_df['name'] = Name_
km_df['light'] = Light_
km_df['PAC'] = PAC[img2county]
km_df.to_csv('Data/km/Data2017_PCA80_KMeans.csv', index = False, header = True)

## National

In [ ]:
# 取出数据
national_df_lst = extract_and_split(km_df, 'national')

#### straitified KMeans

In [ ]:
national_nums = [5, 25, 60] # self-identified according to experience

In [ ]:
# 分类聚类
national_df = cls_cluster(national_df_lst, national_nums)

In [ ]:
national_df.to_csv('Data/km/Data2017_national_PCA80_KMeans90_straitified.csv', index = False, header = True)

#### Connect with siScore

In [ ]:
# 对标siScore
national_hist = generate_hist(national_df, sum(national_nums))
national_grid = generate_grid(national_df)
national_hist.to_csv('Data/hist/hist2017_national_PCA80_KMeans90_straitified.csv', index = False, header = True)
national_grid.to_csv('Data/grid/grid2017_national_PCA80_KMeans90_straitified.csv', index = False, header = True)

## Other Districts

In [ ]:
district_lst_dic = {} 
for name in list(District_dic.keys()):
    print('======={}======'.format(name))
    district_lst_dic[name] = df_lst = extract_and_split(km_df, name)

In [ ]:
### Repeat the process of straitified Kmeans, hist, and grid for each district, respectively